In [1]:
import pandas as pd
import numpy as np
from collections import Counter,defaultdict
import networkx as nx

In [2]:
data = pd.read_csv('NAS_processed_032523.csv')

In [3]:
Exposure_Layer2 = data['Exposure_Layer2']
Outcome_Layer2 = data['Outcome_Layer2']

In [4]:
Exposure_Layer2_dict = defaultdict(dict)
Exposure_Layer2_exposure2node = dict()

for i,k in enumerate(Counter(Exposure_Layer2).keys()):
    Exposure_Layer2_dict[i]['count']=Counter(Exposure_Layer2)[k]
    Exposure_Layer2_dict[i]['property']=k
    Exposure_Layer2_exposure2node[k]=i

In [5]:
Outcome_Layer2_dict = defaultdict(dict)
Outcome_Layer2_outcome2node = dict()
existing_node = len(Exposure_Layer2_exposure2node)-1
for i,k in enumerate(Counter(Outcome_Layer2).keys()):
    if k not in Counter(Exposure_Layer2).keys():
        node = existing_node+1
        existing_node = node
        Outcome_Layer2_dict[node]['count']=Counter(Outcome_Layer2)[k]
        #Outcome_Layer1_dict[node]['property1']='Outcome'
        Outcome_Layer2_dict[node]['property']=k
        Outcome_Layer2_outcome2node[k] = node

In [6]:
#combine all nodes **
layer2_2nodes = {**Outcome_Layer2_outcome2node,**Exposure_Layer2_exposure2node}

In [7]:
layer2_nodes2name = dict()

for k in layer2_2nodes.keys():
    layer2_nodes2name[layer2_2nodes[k]]=k

In [8]:
Exposure_Layer2_nodes = [(k,Exposure_Layer2_dict[k]) for k in Exposure_Layer2_dict.keys()]

Outcome_Layer2_nodes = [(k,Outcome_Layer2_dict[k]) for k in Outcome_Layer2_dict.keys()]

In [9]:
#Node frequency count, layer2
#Exposure_Layer2_dict

In [10]:
adj_list = set()

for i,v in enumerate(Exposure_Layer2):
    e_node = layer2_2nodes[v]
    o_node = layer2_2nodes[Outcome_Layer2[i]]
    if (e_node,o_node) not in adj_list:
        adj_list.add((e_node,o_node))

#how many edges
#adj_list
len(adj_list)

130

In [11]:
node_list = defaultdict(list)

for a,b in list(adj_list):
    node_list[a].append(b)

#path of where each node can go
#node_list

## hypothesis generation

In [12]:
mediate_pool = []
visited = set()
for i in range(91): #number of node-91
    if i not in visited:
        tovisit = [(i,str(i),[i])]
        visited.add(i)
        while tovisit:
            n,p,temp = tovisit.pop(0)
            if n not in node_list.keys():
                #print(p) 
                mediate_pool.append(p)
            else:
                for nn in node_list[n]:
                    if nn not in temp:
                        newtemp=temp+[nn]
                        visited.add(nn)
                        tovisit.append((nn,p+'->'+str(nn),newtemp))

In [13]:
mediation_name_pool = []
for st in mediate_pool:
    li = st.split('->')
    if len(li)>=3:
        #print(st) #print mediation pool node numbers
        fulname = ''
        for vv in li:
            fulname+='->'+layer2_nodes2name[int(vv)]
        mediation_name_pool.append(fulname)

In [14]:
#mediation pool
mediation_name_pool_ = [v[2:] for v in mediation_name_pool]

for v in mediation_name_pool_:
    print(v)

air pollution->mitochondrial DNA->lung function
air pollution->DNAmethylation->blood pressure
air pollution->DNAmethylation->mortality
air pollution->DNAmethylation->lung function
air pollution->DNAmethylation->incident cancer
air pollution->mitochondrial DNA->leukocyte telomere length->lung function
air pollution->mitochondrial DNA->leukocyte telomere length->incident cancer
air pollution->mitochondrial DNA->DNAmethylation->blood pressure
air pollution->mitochondrial DNA->DNAmethylation->mortality
air pollution->mitochondrial DNA->DNAmethylation->lung function
air pollution->mitochondrial DNA->DNAmethylation->incident cancer
air pollution->DNAmethylation->leukocyte telomere length->lung function
air pollution->DNAmethylation->leukocyte telomere length->incident cancer
air pollution->DNAmethylation->mitochondrial DNA->lung function
air pollution->metabolites->DNAmethylation->blood pressure
air pollution->metabolites->DNAmethylation->mortality
air pollution->metabolites->DNAmethylation-

In [15]:
Layer2_mediationpool = pd.DataFrame(data = mediation_name_pool_,columns=['path'])


In [16]:
Layer2_mediationpool.to_csv('Layer2_mediationpool.csv')

## network centrality analysis

In [17]:
#network python graph, needs to be tuple. 
Exposure_Layer2_nodes = [(k,Exposure_Layer2_dict[k]) for k in Exposure_Layer2_dict.keys()]

#Exposure_Layer2_nodes
Outcome_Layer2_nodes = [(k,Outcome_Layer2_dict[k]) for k in Outcome_Layer2_dict.keys()]
G = nx.DiGraph()
G.add_nodes_from(Outcome_Layer2_nodes)
G.add_nodes_from(Exposure_Layer2_nodes)
G.add_edges_from(list(adj_list))

#len(G.edges())

In [18]:
from collections import defaultdict

data = []
for k in dict(G.in_degree()).keys():
    data.append((layer2_nodes2name[k],G.in_degree()[k],G.out_degree()[k],nx.in_degree_centrality(G)[k],nx.out_degree_centrality(G)[k],nx.betweenness_centrality(G)[k],nx.closeness_centrality(G)[k]))

In [19]:
df = pd.DataFrame(data, columns =['Node name', 'In degree', 'Out degree','In degree centrality','Out degree centrality','Betweeness centrality','Closeness Centranilty'])

In [20]:
df

,Node name,In degree,Out degree,In degree centrality,Out degree centrality,Betweeness centrality,Closeness Centranilty
0,white blood cell counts,1,0,0.011905,0.000000,0.0,0.011905
1,blood pressure,4,0,0.047619,0.000000,0.0,0.164130
2,cognitive function,8,0,0.095238,0.000000,0.0,0.112875
3,serum uric acid,1,0,0.011905,0.000000,0.0,0.015873
4,incident chronic kidney disease,1,0,0.011905,0.000000,0.0,0.015873
...,...,...,...,...,...,...,...
80,BMI,0,1,0.000000,0.011905,0.0,0.000000
81,openness to experiences,0,1,0.000000,0.011905,0.0,0.000000
82,metabolic syndrome component,0,2,0.000000,0.023810,0.0,0.000000
83,stressful life events,0,1,0.000000,0.011905,0.0,0.000000


In [21]:
df.to_csv('Layer2_networkstatistics.csv',sep='\t') 

In [22]:
#top nodes sorted by in degree centrality
INDEGREE=df.sort_values(by=['In degree'],ascending=False)
INDEGREE.head(7)
#INDEGREE.to_csv('Layer2_INDEGREE.csv',sep='\t') 

,Node name,In degree,Out degree,In degree centrality,Out degree centrality,Betweeness centrality,Closeness Centranilty
48,DNAmethylation,20,6,0.238095,0.071429,0.020009,0.241071
12,mortality,8,0,0.095238,0.000000,0.000000,0.215030
2,cognitive function,8,0,0.095238,0.000000,0.000000,0.112875
8,lung function,8,0,0.095238,0.000000,0.000000,0.184524
15,inflammation biomarkers,5,0,0.059524,0.000000,0.000000,0.064815
18,incident abnormal cardiac conductivity,4,0,0.047619,0.000000,0.000000,0.058333
43,metabolites,4,1,0.047619,0.011905,0.000502,0.058333


In [23]:
#top nodes sorted by in betweeness centrality
BETWEENNESS=df.sort_values(by=['Betweeness centrality'],ascending=False)
BETWEENNESS.head(7)
#BETWEENNESS.to_csv('Layer2_BETWEENNESS.csv',sep='\t')

,Node name,In degree,Out degree,In degree centrality,Out degree centrality,Betweeness centrality,Closeness Centranilty
48,DNAmethylation,20,6,0.238095,0.071429,0.020009,0.241071
44,bone lead,2,15,0.023810,0.178571,0.004375,0.023810
40,air pollution,1,25,0.011905,0.297619,0.004016,0.011905
62,depression,1,2,0.011905,0.023810,0.003155,0.027211
70,cholesterol level,3,2,0.035714,0.023810,0.002940,0.035714
47,toenail metals,1,3,0.011905,0.035714,0.001147,0.011905
46,blood lead,1,9,0.011905,0.107143,0.000645,0.011905


In [24]:
#top nodes sorted by in outdegree centrality
OUTDEGREE=df.sort_values(by=['Out degree'],ascending=False)
OUTDEGREE.head(7)
#OUTDEGREE.to_csv('Layer2_OUTDEGREE.csv',sep='\t')  

,Node name,In degree,Out degree,In degree centrality,Out degree centrality,Betweeness centrality,Closeness Centranilty
40,air pollution,1,25,0.011905,0.297619,0.004016,0.011905
44,bone lead,2,15,0.023810,0.178571,0.004375,0.023810
42,temperature,0,9,0.000000,0.107143,0.000000,0.000000
46,blood lead,1,9,0.011905,0.107143,0.000645,0.011905
48,DNAmethylation,20,6,0.238095,0.071429,0.020009,0.241071
56,diet,0,6,0.000000,0.071429,0.000000,0.000000
53,optimism,0,5,0.000000,0.059524,0.000000,0.000000


## Top 2 communities

In [25]:
from networkx.algorithms import community

In [26]:
c = community.greedy_modularity_communities(G)
#c

In [27]:
for v in c[0]:
    print(layer2_nodes2name[v])

air pollution
incident metabolic syndrome
metabolic syndrome components
metabolites
temperature
fasting blood glucose
microRNA
perceived stress
arterial stiffness
intima-media thickness of the common carotid artery
road proximity
oxidative DNA damage
serum uric acid
incident chronic kidney disease
blood urea nitrogen
glomerular filtration rate
hemoglobin concentration
intraocular pressure
leukocyte distribution


In [28]:
for v in c[1]:
    print(layer2_nodes2name[v])

pessimism
asthma
allergic phenotypes
mitochondrial DNA
DNAmethylation
leukocyte telomere length
leukocyte telomere length change rate
optimism
drug use
relative humidity
endothelial dysfunction markers
lung function
anxiety
hostility
happiness
life satisfaction
tocopherol levels
incident cancer
